In [ ]:
!pip install -q transformers bitsandbytes faiss-cpu sentence-transformers y


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import zipfile
from google.colab import files

with zipfile.ZipFile("/content/archive.zip", "r") as zip_ref:
    zip_ref.extractall("dataset")

In [ ]:
df=pd.read_csv("/content/kcc_dataset.csv")
df.head()

<ipython-input-4-edbfaf140d6b>:1: DtypeWarning: Columns (0,1,5,6,7,8,9,10,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv("/content/kcc_dataset.csv")


,BlockName,Category,Year,Month,Day,Crop,DistrictName,QueryType,Season,Sector,StateName,QueryText,KccAns
0,0,0,2006.0,1.0,17.0,1275,SAGAR,99,RABI,HORTICULTURE,MADHYA PRADESH,how to control flower drop in bottelgourd,spray planofix4mlpump
1,0,0,2006.0,1.0,17.0,964,SAGAR,Disease Management,RABI,ANIMAL HUSBANDRY,MADHYA PRADESH,how tyo control diseases in buffalo,NaN
2,0,0,2006.0,1.0,17.0,1279,SAGAR,76,RABI,HORTICULTURE,MADHYA PRADESH,how to control fruit borer in brinjal,should be spray profenophos 35mlpump
3,0,0,2006.0,1.0,17.0,1064,SAGAR,3,RABI,AGRICULTURE,MADHYA PRADESH,how to control of yellow moisac in moong,should be spray metasystox 35mlpump
4,0,0,2006.0,1.0,17.0,1279,DAMOH,76,RABI,HORTICULTURE,MADHYA PRADESH,how to control white fly in brinjal,should be spray metasystox 35mlpump


In [ ]:
df.isnull()

,BlockName,Category,Year,Month,Day,Crop,DistrictName,QueryType,Season,Sector,StateName,QueryText,KccAns
0,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,True
2,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
977339,True,True,True,True,True,True,True,True,True,True,True,True,True
977340,True,True,True,True,True,True,True,True,True,True,True,True,True
977341,True,True,True,True,True,True,True,True,True,True,True,True,True
977342,True,True,True,True,True,True,True,True,True,True,True,True,True


In [ ]:
def df['BlockName']
del df['Category']
df.head()

,Year,Month,Day,Crop,DistrictName,QueryType,Season,Sector,StateName,QueryText,KccAns
0,2006.0,1.0,17.0,1275,SAGAR,99,RABI,HORTICULTURE,MADHYA PRADESH,how to control flower drop in bottelgourd,spray planofix4mlpump
1,2006.0,1.0,17.0,964,SAGAR,Disease Management,RABI,ANIMAL HUSBANDRY,MADHYA PRADESH,how tyo control diseases in buffalo,NaN
2,2006.0,1.0,17.0,1279,SAGAR,76,RABI,HORTICULTURE,MADHYA PRADESH,how to control fruit borer in brinjal,should be spray profenophos 35mlpump
3,2006.0,1.0,17.0,1064,SAGAR,3,RABI,AGRICULTURE,MADHYA PRADESH,how to control of yellow moisac in moong,should be spray metasystox 35mlpump
4,2006.0,1.0,17.0,1279,DAMOH,76,RABI,HORTICULTURE,MADHYA PRADESH,how to control white fly in brinjal,should be spray metasystox 35mlpump


In [ ]:

def chunk_text(df, chunk_size=500, overlap=100):
    chunks = []

    for index, row in df.iterrows():
        text = f"{row.get('Crop', '')} {row.get('DistrictName', '')} {row.get('Season', '')} {row.get('Sector', '')} {row.get('StateName', '')} {row.get('QueryText', '')} {row.get('KccAns', '')}"
        for i in range(0, len(text), chunk_size - overlap):
            chunk = text[i:i + chunk_size]
            if chunk.strip():
                chunks.append(chunk)
    return chunks


chunks = chunk_text(df)
print(f"Split into {len(chunks)} chunks")


Split into 977541 chunks


In [ ]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import torch


embedding_model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = embedding_model.encode(chunks, convert_to_tensor=True)
embeddings_np = embeddings.cpu().detach().numpy()

dimension = embeddings_np.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings_np)

print("Chunks embedded and added to FAISS")


Chunks embedded and added to FAISS


In [ ]:
#pip install --upgrade transformers

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch
from huggingface_hub import login
import os
import bitsandbytes

login(token="HF_TOKEN")

model_id = "google/gemma-3-1b-it"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)

tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    quantization_config=bnb_config,
    trust_remote_code=True
)



tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/899 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.00G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

In [ ]:
def get_context(query, top_k):
    with torch.no_grad():
        try:
            query_embedding = embedding_model.encode([query], convert_to_tensor=True)[0]
        except RuntimeError as e:
            print(f"Error embedding query: '{query}'")
            raise e

    D, I = index.search(np.array([query_embedding.cpu().numpy()], dtype=np.float32), top_k)

    return "\n".join([chunks[i] for i in I[0]])


def generate_answer(query):
    context = get_context(query,5)
    prompt = f"""Answer the question as from the provided context, make sure to provide all the methods/points, don't provide the wrong answer and don't repeat yourself \n\n
    Context:
    {context}

    Question: {query}
    Answer:"""

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=150,
            temperature=0.8,
            do_sample=True,
            top_p=0.95,
            top_k=50,
            pad_token_id=tokenizer.eos_token_id
        )

    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer.split("Answer:")[-1].strip()




query = "How to manage drought stress in groundnut cultivation?"
print(generate_answer(query))

The context provides the following methods/points for managing drought stress in groundnut cultivation:

1.  Applying Chlorpyriphos 4L (2g/kg) with irrigation and water.
2.  Using Manuring and water management.
3.  Using Rhizobium and PSB culture.
4.  Using 1653 Ganganagar Rabia Agriculture.
5.  Using varieties of groundnut and its seed treatment.
6.  Applying Chlorpyriphos 20 EC 450 ML per hectare with irrigation and water.
7.  Applying 1653 Chandra and seed treatment with water.
8.  Using 1653 Rajasthan


In [ ]:
model.save_pretrained("kcc_chatbot_gemma")
tokenizer.save_pretrained("kcc_chatbot_gemma")

('kcc_chatbot_gemma/tokenizer_config.json',
 'kcc_chatbot_gemma/special_tokens_map.json',
 'kcc_chatbot_gemma/tokenizer.model',
 'kcc_chatbot_gemma/added_tokens.json',
 'kcc_chatbot_gemma/tokenizer.json')

In [ ]:
!zip -r kcc_chatbot_gemma.zip kcc_chatbot_gemma

  adding: kcc_chatbot_gemma/ (stored 0%)
  adding: kcc_chatbot_gemma/model.safetensors (deflated 16%)
  adding: kcc_chatbot_gemma/generation_config.json (deflated 34%)
  adding: kcc_chatbot_gemma/added_tokens.json (stored 0%)
  adding: kcc_chatbot_gemma/tokenizer.model (deflated 52%)
  adding: kcc_chatbot_gemma/tokenizer_config.json (deflated 96%)
  adding: kcc_chatbot_gemma/config.json (deflated 56%)
  adding: kcc_chatbot_gemma/tokenizer.json (deflated 83%)
  adding: kcc_chatbot_gemma/special_tokens_map.json (deflated 77%)


In [ ]:
import pickle

faiss.write_index(index, "faiss_index.index")

with open("chunks.pkl", "wb") as f:
    pickle.dump(chunks, f)


In [ ]:
!zip saved_embeddings.zip faiss_index.index chunks.pkl
from google.colab import files
files.download("saved_embeddings.zip")

  adding: faiss_index.index (deflated 54%)
  adding: chunks.pkl (deflated 88%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>